In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("data.csv")
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


 Переименовать поля (conversion -> target, offer -> treatment) 

In [2]:
df = df.rename(columns={'conversion': 'target', 'offer':'treatment'})

In [3]:
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


 привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [4]:
df['treatment'].value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: treatment, dtype: int64

In [5]:
df['treatment'] = df['treatment'].replace({'Buy One Get One':1, 'Discount':1, 'No Offer':0})

In [6]:
df['treatment'].value_counts()

1    42694
0    21306
Name: treatment, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   treatment      64000 non-null  int64  
 8   target         64000 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 4.4+ MB


In [8]:
df['zip_code'].value_counts()

Surburban    28776
Urban        25661
Rural         9563
Name: zip_code, dtype: int64

In [9]:
df['zip_code'] = df['zip_code'].replace({'Surburban':1, 'Urban':2, 'Rural':3})

In [10]:
df['channel'].value_counts()

Web             28217
Phone           28021
Multichannel     7762
Name: channel, dtype: int64

In [11]:
df['channel'] = df['channel'].replace({'Web':1, 'Phone':2, 'Multichannel':3})

In [23]:
df_model_bogo = df.copy().loc[df.treatment <=0].reset_index(drop=True)
df_model_discount = df.copy().loc[df.treatment >=0].reset_index(drop=True)

In [26]:
from sklearn.model_selection import train_test_split

features = ['recency', 'history', 'used_discount', 'used_bogo', 'zip_code', 'is_referral', 'channel', 'channel']#, 'treatment']
target = 'target'
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=df['treatment'])

In [13]:
from sklearn.model_selection import train_test_split

features = ['recency', 'history', 'used_discount', 'used_bogo', 'zip_code', 'is_referral', 'channel', 'channel', 'treatment']
target = 'target'
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
df[]

In [17]:
pip install scikit-uplift==0.2.0

Note: you may need to restart the kernel to use updated packages.


In [15]:
models_results = {
    'approach': [],
    'uplift@30%': []
}

In [19]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11434 sha256=7cd493a25f301b03c0ec370c685c715f468f1bd08cb65e344c1b46208a17aa7e
  Stored in directory: c:\users\sheno\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying



In [22]:
treat_val =  X_train.loc['treatment']

KeyError: 'treatment'

In [21]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

# sklift поддерживает любые модели, 
# которые удовлетворяют соглашениями scikit-learn
# Для примера воспользуемся catboost
from catboost import CatBoostClassifier


sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, estimator_fit_params={'features': features})

uplift_sm = sm.predict(X_test)

sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, strategy='by_group', k=0.3)
print(f'uplift@30%: {sm_score:.4f}')

models_results['approach'].append('SoloModel')
models_results['uplift@30%'].append(sm_score)

# Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
sm_trmnt_preds = sm.trmnt_preds_
# И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
sm_ctrl_preds = sm.ctrl_preds_

# Отрисуем распределения вероятностей и их разность (uplift)
plot_uplift_preds(trmnt_preds=sm_trmnt_preds, ctrl_preds=sm_ctrl_preds);

TypeError: fit() missing 1 required positional argument: 'treatment'